# Batch Analysis with VAAS Using a Folder-Based Workflow

In real-world scenarios, analysts rarely inspect a single image in isolation.
More commonly, they work with collections of images that need to be screened,
ranked, or triaged.

The current VAAS API operates on one image at a time.
This notebook demonstrates how to perform batch-style analysis by combining
VAAS with a simple and explicit Python workflow.

The emphasis here is clarity and control rather than automation.


## Installation

Install VAAS if it is not already available in the environment.


In [ ]:
!pip install vaas==0.1.7

In [ ]:
from vaas.inference.pipeline import VAASPipeline
from PIL import Image
import requests
from io import BytesIO

## Loading the VAAS Pipeline

We initialize the VAAS pipeline once and reuse it across multiple images.
This avoids unnecessary reloading of model weights.


In [ ]:
pipeline = VAASPipeline.from_pretrained(
    "OBA-Research/vaas-v1-df2023",
    device="cpu",
    alpha=0.5,
)

## Defining an Image Collection

For ease of execution, we define a small collection of image URLs
hosted in the VAAS GitHub repository.

In practice, this list could be generated from a local folder,
a database query, or an external dataset.

In [ ]:
image_urls = [
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/examples/images/COCO_DF_C110B00000_00539519.jpg",
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/examples/images/COCO_DF_E000B00011_00834527.jpg",
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/examples/images/COCO_DF_E000B04010_00863031.jpg",
]

## Loading Images from URLs

To keep the main loop readable, we define a small helper function
for loading images from URLs.


In [ ]:
def load_image_from_url(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")


## Running Batch-Style Inference

We now iterate over the image collection and apply VAAS inference
to each image individually.

This explicit loop makes it easy to inspect intermediate results
and customize downstream behavior.


In [ ]:
results = []

for url in image_urls:
    image = load_image_from_url(url)
    output = pipeline(image)

    results.append(
        {
            "image_url": url,
            "S_F": output["S_F"],
            "S_P": output["S_P"],
            "S_H": output["S_H"],
        }
    )


## Inspecting the Results

Below we print the anomaly scores for each image in the collection.


## Practical Interpretation

In batch-style workflows, the hybrid score (S_H) is often used to:

- rank images by degree of visual inconsistency,
- identify a subset of images for closer inspection,
- filter large collections before manual review.

This approach scales naturally as long as memory constraints
are respected.


## Visualizing Selected Results

After identifying images of interest, visualization can be applied
selectively rather than to every image.


In [ ]:
selected_image = load_image_from_url(image_urls[0])

pipeline.visualize(
    image=selected_image,
    save_path="vaas_selected_visualization.png",
    mode="all",
    threshold=0.5,
)


In [ ]:
from IPython.display import Image as IPImage, display
display(IPImage("vaas_selected_visualization.png"))


## Why This Design Is Intentional

Explicit batch workflows:
- avoid hidden complexity,
- allow fine-grained control,
- and make performance trade-offs visible.

For many forensic and research workflows, this level of control
is preferable to opaque batch abstractions.


## Summary

In this notebook, we:
- demonstrated how to analyze multiple images using VAAS,
- implemented batch-style inference with an explicit loop,
- and discussed practical usage patterns and limitations.

The next notebook will build on this approach by ranking images
according to their anomaly scores.

check it out: [08_ranking_images_by_visual_suspicion.ipynb](https://colab.research.google.com/drive/18D4eV_fgomOIrxsyP_U__HYrTl-ZtC8e?usp=sharing)
